In [3]:
from group_formations import get_xy_data_grouped_by_formation

xy_by_formation = get_xy_data_grouped_by_formation()

Processed home team for match: 1. FC Köln:FC Bayern München with lineup 4-2-3-1
Processed away team for match: 1. FC Köln:FC Bayern München with lineup 4-2-3-1
Processed home team for match: VfL Bochum 1848:Bayer 04 Leverkusen with lineup 4-3-3 Double-6
Processed away team for match: VfL Bochum 1848:Bayer 04 Leverkusen with lineup 4-2-3-1
Processed home team for match: Fortuna Düsseldorf:FC St. Pauli with lineup 3-4-1-2
Processed away team for match: Fortuna Düsseldorf:FC St. Pauli with lineup 3-3-2-2
Processed home team for match: Fortuna Düsseldorf:1. FC Kaiserslautern with lineup 4-4-2 Double-6 offensive Outliner
Processed away team for match: Fortuna Düsseldorf:1. FC Kaiserslautern with lineup 4-2-3-1
Processed home team for match: Fortuna Düsseldorf:1. FC Nürnberg with lineup 4-2-3-1
Processed away team for match: Fortuna Düsseldorf:1. FC Nürnberg with lineup 4-1-3-2
Processed home team for match: Fortuna Düsseldorf:SSV Jahn Regensburg with lineup 4-2-3-1
Processed away team for m

In [4]:
print("Formations:")
xy_by_formation.keys()

Formations:


dict_keys(['4-2-3-1', '4-3-3 Double-6', '3-4-1-2', '3-3-2-2', '4-4-2 Double-6 offensive Outliner', '4-1-3-2'])

In [5]:
xy_by_formation['4-2-3-1'][0]

[47.37,
 0.25,
 13.08,
 -13.71,
 18.84,
 -4.49,
 17.26,
 20.85,
 20.16,
 7.08,
 10.39,
 -3.39,
 11.37,
 5.18,
 1.4,
 13.05,
 0.44,
 -15.57,
 6.9,
 5.12,
 -2.12,
 -7.09]

In [ ]:
# TODO: Plot data for each formation